# Tarea de clasificación de texto

Como hemos mencionado, nos centraremos en una tarea sencilla de clasificación de texto basada en el dataset **AG_NEWS**, que consiste en clasificar titulares de noticias en una de las 4 categorías: Mundo, Deportes, Negocios y Ciencia/Tecnología.

## El Dataset

Este dataset está integrado en el módulo [`torchtext`](https://github.com/pytorch/text), por lo que podemos acceder a él fácilmente.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Aquí, `train_dataset` y `test_dataset` contienen colecciones que devuelven pares de etiqueta (número de clase) y texto respectivamente, por ejemplo:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

Entonces, imprimamos los primeros 10 nuevos titulares de nuestro conjunto de datos:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Debido a que los conjuntos de datos son iteradores, si queremos usar los datos varias veces, necesitamos convertirlos a una lista:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Tokenización

Ahora necesitamos convertir el texto en **números** que puedan representarse como tensores. Si queremos una representación a nivel de palabras, debemos hacer dos cosas:  
* usar un **tokenizador** para dividir el texto en **tokens**  
* construir un **vocabulario** de esos tokens.  


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

Usando vocabulario, podemos codificar fácilmente nuestra cadena tokenizada en un conjunto de números:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Representación de texto con Bolsa de Palabras

Debido a que las palabras representan significado, a veces podemos entender el significado de un texto simplemente observando las palabras individuales, sin importar su orden en la oración. Por ejemplo, al clasificar noticias, palabras como *clima*, *nieve* probablemente indiquen *pronóstico del tiempo*, mientras que palabras como *acciones*, *dólar* podrían corresponder a *noticias financieras*.

La representación vectorial **Bolsa de Palabras** (BoW) es la representación vectorial tradicional más utilizada. Cada palabra está vinculada a un índice del vector, y el elemento del vector contiene el número de veces que una palabra aparece en un documento dado.

![Imagen que muestra cómo se representa en memoria una representación vectorial de bolsa de palabras.](../../../../../lessons/5-NLP/13-TextRep/images/bag-of-words-example.png) 

> **Nota**: También puedes pensar en BoW como la suma de todos los vectores codificados en uno a uno para las palabras individuales en el texto.

A continuación, se muestra un ejemplo de cómo generar una representación de bolsa de palabras utilizando la biblioteca de Python Scikit Learn:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Para calcular el vector de bolsa de palabras a partir de la representación vectorial de nuestro conjunto de datos AG_NEWS, podemos usar la siguiente función:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Nota:** Aquí estamos utilizando la variable global `vocab_size` para especificar el tamaño predeterminado del vocabulario. Dado que a menudo el tamaño del vocabulario es bastante grande, podemos limitar el tamaño del vocabulario a las palabras más frecuentes. Intenta reducir el valor de `vocab_size` y ejecutar el código a continuación, y observa cómo afecta la precisión. Deberías esperar una disminución en la precisión, pero no dramática, en lugar de un mayor rendimiento.


## Entrenando el clasificador BoW

Ahora que hemos aprendido cómo construir la representación de Bag-of-Words para nuestro texto, vamos a entrenar un clasificador sobre esta base. Primero, necesitamos convertir nuestro conjunto de datos para el entrenamiento de tal manera que todas las representaciones vectoriales posicionales se conviertan en representaciones de Bag-of-Words. Esto se puede lograr pasando la función `bowify` como el parámetro `collate_fn` al `DataLoader` estándar de torch:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Ahora definamos una red neuronal clasificador simple que contiene una capa lineal. El tamaño del vector de entrada es igual a `vocab_size`, y el tamaño de salida corresponde al número de clases (4). Debido a que estamos resolviendo una tarea de clasificación, la función de activación final es `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Ahora definiremos el bucle de entrenamiento estándar de PyTorch. Debido a que nuestro conjunto de datos es bastante grande, para nuestro propósito de enseñanza entrenaremos solo por una época, y a veces incluso por menos de una época (especificar el parámetro `epoch_size` nos permite limitar el entrenamiento). También informaremos la precisión acumulada del entrenamiento durante el proceso; la frecuencia de los informes se especifica utilizando el parámetro `report_freq`.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## BiGrams, TriGrams y N-Grams

Una limitación del enfoque de bolsa de palabras es que algunas palabras forman expresiones de varias palabras. Por ejemplo, la palabra 'hot dog' tiene un significado completamente diferente al de las palabras 'hot' y 'dog' en otros contextos. Si representamos las palabras 'hot' y 'dog' siempre con los mismos vectores, esto puede confundir a nuestro modelo.

Para abordar este problema, las **representaciones N-gram** se utilizan frecuentemente en métodos de clasificación de documentos, donde la frecuencia de cada palabra, bi-palabra o tri-palabra es una característica útil para entrenar clasificadores. En la representación de bigramas, por ejemplo, añadimos todos los pares de palabras al vocabulario, además de las palabras originales.

A continuación, se muestra un ejemplo de cómo generar una representación de bolsa de palabras con bigramas utilizando Scikit Learn:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

La principal desventaja del enfoque N-gram es que el tamaño del vocabulario comienza a crecer extremadamente rápido. En la práctica, necesitamos combinar la representación N-gram con algunas técnicas de reducción de dimensionalidad, como *embeddings*, que discutiremos en la próxima unidad.

Para usar la representación N-gram en nuestro conjunto de datos **AG News**, necesitamos construir un vocabulario especial de ngram:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


Podríamos usar el mismo código mencionado anteriormente para entrenar el clasificador, sin embargo, sería muy ineficiente en términos de memoria. En la próxima unidad, entrenaremos un clasificador de bigramas utilizando embeddings.

> **Nota:** Solo puedes conservar aquellos ngrams que aparezcan en el texto más veces que el número especificado. Esto asegurará que los bigramas poco frecuentes sean omitidos y reducirá significativamente la dimensionalidad. Para lograr esto, ajusta el parámetro `min_freq` a un valor más alto y observa cómo cambia la longitud del vocabulario.


## Frecuencia de Términos e Inversa Frecuencia de Documentos TF-IDF

En la representación BoW, las ocurrencias de palabras tienen el mismo peso, sin importar la palabra en sí. Sin embargo, está claro que las palabras frecuentes, como *a*, *en*, etc., son mucho menos importantes para la clasificación que los términos especializados. De hecho, en la mayoría de las tareas de PLN, algunas palabras son más relevantes que otras.

**TF-IDF** significa **frecuencia de términos–inversa frecuencia de documentos**. Es una variación de bolsa de palabras, donde en lugar de un valor binario 0/1 que indica la aparición de una palabra en un documento, se utiliza un valor de punto flotante relacionado con la frecuencia de aparición de la palabra en el corpus.

Más formalmente, el peso $w_{ij}$ de una palabra $i$ en el documento $j$ se define como:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
donde
* $tf_{ij}$ es el número de ocurrencias de $i$ en $j$, es decir, el valor BoW que hemos visto antes
* $N$ es el número de documentos en la colección
* $df_i$ es el número de documentos que contienen la palabra $i$ en toda la colección

El valor TF-IDF $w_{ij}$ aumenta proporcionalmente al número de veces que una palabra aparece en un documento y se ajusta por el número de documentos en el corpus que contienen la palabra, lo que ayuda a compensar el hecho de que algunas palabras aparecen con más frecuencia que otras. Por ejemplo, si la palabra aparece en *todos* los documentos de la colección, $df_i=N$, y $w_{ij}=0$, y esos términos serían completamente ignorados.

Puedes crear fácilmente una vectorización TF-IDF de texto utilizando Scikit Learn:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Conclusión

Sin embargo, aunque las representaciones TF-IDF asignan un peso de frecuencia a diferentes palabras, no son capaces de representar el significado ni el orden. Como dijo el famoso lingüista J. R. Firth en 1935: “El significado completo de una palabra siempre es contextual, y ningún estudio del significado fuera del contexto puede tomarse en serio”. Más adelante en el curso aprenderemos cómo capturar información contextual del texto utilizando modelos de lenguaje.



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Aunque nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automatizadas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse como la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
